In [3]:
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
import gcsfs
import json
import os

CLIENT_SECRET_FILE = "client_secrets.json"
TOKEN_FILE = "token1.json"
SCOPES = ["https://www.googleapis.com/auth/cloud-platform"]

# Step 1: Try to load existing token
creds = None
if os.path.exists(TOKEN_FILE):
    creds = Credentials.from_authorized_user_file(TOKEN_FILE, SCOPES)

# Step 2: If no (valid) creds, do manual auth
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(CLIENT_SECRET_FILE, SCOPES)
        
        # Ensure the correct redirect URI is used
        flow.redirect_uri = 'http://localhost:8080'  # Ensure this matches what you set in GCP console

        auth_url, _ = flow.authorization_url(prompt='consent')

        print("🔗 Go to this URL:\n", auth_url)
        code = input("📋 Paste the authorization code here: ")
        flow.fetch_token(code=code)
        creds = flow.credentials

        # Save token for future use
        with open(TOKEN_FILE, "w") as token:
            token.write(creds.to_json())

# Step 3: Use GCSFS with your personal creds
fs = gcsfs.GCSFileSystem(token=creds)

# Example: list files in the bucket
print("✅ Listing files in bucket...")
print(fs.ls("waymo_open_dataset_motion_v_1_3_0"))


🔗 Go to this URL:
 https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=6717835038-srvt7mfdgh66ubjj5636uk0d7e2ipken.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform&state=gFMHVDnDA6o58zhatCeUCndhzupErS&prompt=consent&access_type=offline
✅ Listing files in bucket...
['waymo_open_dataset_motion_v_1_3_0/uncompressed']
